In [1]:
! pip install transformers
! pip install datasets
from datasets import load_dataset
import random
random.seed(42)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00


In [2]:
train_data = load_dataset("yangwang825/rte", split="train")
dev_data = load_dataset("yangwang825/rte", split="validation")
test_data = load_dataset("yangwang825/rte", split="test")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [3]:
print(train_data)

Dataset({
    features: ['text1', 'text2', 'label', 'idx', 'label_text'],
    num_rows: 2490
})


In [4]:
from transformers import BertTokenizer,BertModel

In [5]:
tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-mini")
model = BertModel.from_pretrained("prajjwal1/bert-mini")

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

In [6]:
train_tokens = tokenizer(train_data["text1"], train_data["text2"], truncation=True, padding=True,max_length=512, return_tensors="pt")
dev_tokens = tokenizer(dev_data["text1"], dev_data["text2"], truncation=True, padding=True,max_length=512, return_tensors="pt")
test_tokens = tokenizer(test_data["text1"], test_data["text2"], truncation=True, padding=True,max_length=512, return_tensors="pt")

In [7]:
import torch

In [8]:
from torch.utils.data import DataLoader, TensorDataset
train_dataset = TensorDataset(train_tokens["input_ids"], train_tokens["token_type_ids"], train_tokens["attention_mask"], torch.tensor(train_data["label"]))
dev_dataset = TensorDataset(dev_tokens["input_ids"], dev_tokens["token_type_ids"], dev_tokens["attention_mask"], torch.tensor(dev_data["label"]))
test_dataset = TensorDataset(test_tokens["input_ids"], test_tokens["token_type_ids"], test_tokens["attention_mask"], torch.tensor(test_data["label"]))
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [9]:
from transformers import BertModel
import numpy as np

In [10]:
import torch
class CustomBert(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.bert = BertModel.from_pretrained("prajjwal1/bert-mini")        # Invoke the constructor of the parent class
    self.l1 = torch.nn.Linear(256, 2,bias = True)

  def forward(self, inp_ids,tok_typs,att_masks):
    a = self.bert(input_ids=inp_ids, token_type_ids=tok_typs,attention_mask=att_masks)
    z = self.l1(a.pooler_output)
    return z

device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")
model = CustomBert().to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

In [11]:
num_epochs = 10
best_perf_dict = {"metric": 0, "epoch": 0}
for epoch in range(1,num_epochs+1):
  print(f"\nEpoch {epoch}")
  accuracies = []
  loss_mat =[]
  for batch in train_dataloader:
    model.train()
    optimizer.zero_grad()
    input_ids,token_type_ids,attention_mask,labels = batch
    outputs = model(input_ids,token_type_ids,attention_mask)
    pred_lab = torch.argmax(outputs,dim=1)
    loss = loss_fn(outputs,labels)
    loss.backward() # computing the gradients
    optimizer.step()  # Performs the optimization
    temp_accuracy = 0
    for k in range(len(pred_lab)):
      if pred_lab[k] == labels[k]:
        temp_accuracy = temp_accuracy + 1
    temp_accuracy = (temp_accuracy)/(len(pred_lab))
    accuracies.append(temp_accuracy)
    loss_mat.append(loss.detach().numpy())
  print(f"Average Accuracy: {np.mean(accuracies)}")
  print(f"Average Loss: {np.mean(loss_mat)}")
    # Evaluation Loop
  dev_accuracies = []
  for batch in dev_dataloader:
      model.eval()
      input_ids,token_type_ids,attention_mask,labels = batch
      with torch.no_grad():
          # out = model(inp.to(device))
          outputs = model(input_ids,token_type_ids,attention_mask)
          pred_lab = torch.argmax(outputs,dim=1)
          temp_accuracy = 0
          for k in range(len(pred_lab)):
            if pred_lab[k] == labels[k]:
              temp_accuracy = temp_accuracy + 1
          temp_accuracy = (temp_accuracy)/(len(pred_lab))
          dev_accuracies.append(temp_accuracy)
  print(f"Dev Accuracy: {np.mean(dev_accuracies)}")
  dev_f1 = np.mean(dev_accuracies)
  # Update the `best_perf_dict` if the best dev performance seen
  # so far is beaten
  if dev_f1 > best_perf_dict["metric"]:
      best_perf_dict["metric"] = dev_f1
      best_perf_dict["epoch"]  = epoch
      torch.save({
          "model_param": model.state_dict(),
          "optim_param": optimizer.state_dict(),
          "dev_metric": dev_f1,
          "epoch": epoch
      }, f"./models/{epoch}")



Epoch 1
Average Accuracy: 0.5362980769230768
Average Loss: 0.6898173689842224
Dev Accuracy: 0.5861111111111111

Epoch 2
Average Accuracy: 0.601923076923077
Average Loss: 0.6641821265220642
Dev Accuracy: 0.60625

Epoch 3
Average Accuracy: 0.6556891025641025
Average Loss: 0.6267523169517517
Dev Accuracy: 0.6305555555555555

Epoch 4
Average Accuracy: 0.6870993589743589
Average Loss: 0.5883151888847351
Dev Accuracy: 0.6513888888888889

Epoch 5
Average Accuracy: 0.7186698717948717
Average Loss: 0.5509724617004395
Dev Accuracy: 0.6375

Epoch 6
Average Accuracy: 0.7485576923076923
Average Loss: 0.5172396302223206
Dev Accuracy: 0.6194444444444445

Epoch 7
Average Accuracy: 0.7817307692307692
Average Loss: 0.46845850348472595
Dev Accuracy: 0.6361111111111111

Epoch 8
Average Accuracy: 0.8136217948717949
Average Loss: 0.4221673607826233
Dev Accuracy: 0.6333333333333333

Epoch 9
Average Accuracy: 0.8332532051282052
Average Loss: 0.3845900893211365
Dev Accuracy: 0.6368055555555556

Epoch 10
Avera

In [12]:
best_epoch = best_perf_dict["epoch"]
model_path = f"./models/{best_epoch}"
checkpoint = torch.load(model_path)
test_accuracies = []
for batch in test_dataloader:
    model.eval()
    input_ids,token_type_ids,attention_mask,labels = batch
    with torch.no_grad():
        # out = model(inp.to(device))
        outputs = model(input_ids,token_type_ids,attention_mask)
        pred_lab = torch.argmax(outputs,dim=1)
        temp_accuracy = 0
        for k in range(len(pred_lab)):
          if pred_lab[k] == labels[k]:
            temp_accuracy = temp_accuracy + 1
        temp_accuracy = (temp_accuracy)/(len(pred_lab))
        test_accuracies.append(temp_accuracy)
print(f"Test Accuracy: {np.mean(test_accuracies)}")

Test Accuracy: 0.6590277777777778


In [13]:
import pandas as pd
df = pd.read_csv("hidden_rte.csv")
le = len(df.index)
import random
random_numbers = []
for i in range(le):
    random_numbers.append(random.randint(0,1))
df["label"]=random_numbers.copy()
df["probab_0"]=np.random.rand(le)
df["probab_1"]=np.random.rand(le)
hidden_rte = df.to_dict(orient='list')

In [14]:
hidden_rte_tokens = tokenizer(hidden_rte["text1"], hidden_rte["text2"], truncation=True, padding=True,max_length=512, return_tensors="pt")

In [15]:
hidden_rte_dataset = TensorDataset(hidden_rte_tokens["input_ids"], hidden_rte_tokens["token_type_ids"], hidden_rte_tokens["attention_mask"], torch.tensor(hidden_rte["label"]))

In [16]:
hidden_rte_dataloader = DataLoader(hidden_rte_dataset, batch_size=1)
i = -1
for batch in hidden_rte_dataloader:
    i = i+1
    model.eval()
    input_ids,token_type_ids,attention_mask,labels = batch
    with torch.no_grad():
        # out = model(inp.to(device))
        outputs = model(input_ids,token_type_ids,attention_mask)
        outputs = torch.softmax(outputs,dim = 1)
        # print(outputs)
        df.loc[i, "probab_0"] = outputs[0][0].item()
        df.loc[i, "probab_1"] = outputs[0][1].item()
        pred_lab = torch.argmax(outputs,dim=1)
        df.loc[i, "label"] = pred_lab[0].item()
        # print(pred_lab)


In [17]:
print(df)

                                                 text1  \
0    This church choir sings to the masses as they ...   
1    This church choir sings to the masses as they ...   
2    A woman with a green headscarf, blue shirt and...   
3    A woman with a green headscarf, blue shirt and...   
4    An old man with a package poses in front of an...   
..                                                 ...   
97   A professional swimmer spits water out after s...   
98   The man in the black wetsuit is walking out of...   
99   The man in the black wetsuit is walking out of...   
100   Five girls and two guys are crossing a overpass.   
101   Five girls and two guys are crossing a overpass.   

                                             text2  label  probab_0  probab_1  
0                  The church is filled with song.      0  0.796250  0.203749  
1              A choir singing at a baseball game.      1  0.051314  0.948686  
2                         The woman is very happy.      1  0.09

In [23]:
df.to_csv('rte.csv')

In [18]:
hidden_q_rte ={"text2":["She is prescribing medicine.","He is prescribing medicine.","She is tending to the patient.","He is tending to the patient."],"text1":["The doctor is prescribing medicine.","The doctor is prescribing medicine.","The nurse is tending to the patient.","The nurse is tending to the patient."]}

In [19]:
hidden_q_tokens = tokenizer(hidden_q_rte["text1"], hidden_q_rte["text2"], truncation=True, padding=True,max_length=512, return_tensors="pt")

In [21]:
hidden_q_dataset = TensorDataset(hidden_q_tokens["input_ids"], hidden_q_tokens["token_type_ids"], hidden_q_tokens["attention_mask"])

In [22]:
hidden_q_dataloader = DataLoader(hidden_q_dataset, batch_size=1)
i = -1
for batch in hidden_q_dataloader:
    i = i+1
    model.eval()
    input_ids,token_type_ids,attention_mask = batch
    with torch.no_grad():
        # out = model(inp.to(device))
        outputs = model(input_ids,token_type_ids,attention_mask)
        pred_lab = torch.argmax(outputs,dim=1)
        print(pred_lab)

tensor([0])
tensor([0])
tensor([0])
tensor([0])
